# Calibration backend

In [57]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

def calibrate(file_name, axis, voltage):
    # Check if axis is a string
    if not isinstance(axis, str):
        raise ValueError("Axis must be a string")
    
    # Read the data from the CSV file and skip every other row
    df = pd.read_csv(file_name)
    axis = str(axis)
    axis_dict = {'x': 0, 'y': 1, 'z': 2}
    converted = axis_dict[axis]

    # Define the function for a linear regression line
    def linear_regression(v, a, b):
        return a * v + b
    
    # Initialize empty lists to store params and errors
    params_list = []
    errors_list = []

    for axis in ['x', 'y', 'z']:
        # Fit the linear regression line to the data
        params, covariance = curve_fit(linear_regression, df['volts (V)'], df[f'actual_distance_{axis}(um)'])
        params_list.append(params)
        
        # Errors in slope and y-intercept
        errors = np.sqrt(np.diag(covariance))
        errors_list.append(errors)      

    chosen_params = params_list[converted]
    errors = errors_list[converted]
    
    calibrated_value = chosen_params[0]*voltage + chosen_params[1]
    error_calibrated_value = np.sqrt(errors[0]*voltage + errors[1])
    
    print(f'At the {voltage} V, the calibrated value with uncertainty for {axis} is {calibrated_value:.4f} ± {error_calibrated_value:.4f} µm.')

    return None

## Output

In [64]:
calibrate('stage_data_final.csv', 'y', 50)

At the 50 V, the calibrated value with uncertainty for z is 44.8627 ± 1.0010 µm.
